In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# On-device recommendations with Firebase ML and TensorFlow Lite

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/examples/recommendation/ml/ondevice_recommendation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/lite/examples/recommendation/ml/ondevice_recommendation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

This is the notebook for step 11 of the codelab [**Add recommendations to your app with TensorFlow Lite and Firebase**](https://codelabs.developers.google.com/codelabs/contentrecommendation-android). Before running the code in this notebook, complete steps 1-10 of the codelab to get your app and console projects set up.

This code base provides a toolkit to train an on-device recommendation
tensorflow model with user data collected in your app with Firebase Analytics. This model will then be deployed with Firebase ML to serve movie recommendations in the sample app FireFlix. 

This Notebook shows an end-to-end example that 1) imports Firebase Analytics data from BigQuery 2) preprocesses that data to prepare it for training 3) trains a recommendations model using the data and 4) exports the model in tflite format, ready to use in apps to run inference and serve recommendations.

Since the app we use in the codelab is just a sample app, it doesn't have the usage necessary to generate a significant amount of analytics events. Since training accurate models requires a large amount of data, for the purposes of this codelab and notebook, we will be simulating a larger analytics event store by using the public [movielens](https://grouplens.org/datasets/movielens/) dataset, but you could
adapt the data processing script for your dataset and train your own
recommendation model.

## Prerequisites

Run the cell below to clone the tensorflow recommendations model sample from Github. This is the model we will use, with our analytics training data, to create the recommendations model.

The model uses a Convolutional neural-network encoder (CNN): applying multiple layers of convolutional neural-network to generate an encoding of the user history analytics data. For more details, refer to the [documentation]() for the underlying tensorflow model.

In [ ]:
!git clone https://github.com/tensorflow/examples
%cd examples/lite/examples/recommendation/ml/
!pip install -r requirements.txt
!pip install --upgrade google-cloud-storage google-cloud-bigquery[bqstorage]

fatal: destination path 'examples' already exists and is not an empty directory.
/content/examples/lite/examples/recommendation/ml
Requirement already up-to-date: google-cloud-storage in /usr/local/lib/python3.6/dist-packages (1.31.2)
Requirement already up-to-date: google-cloud-bigquery[bqstorage] in /usr/local/lib/python3.6/dist-packages (2.0.0)


## Set up authentication

In this notebook, we use analytics data from BigQuery to generate training data for our recommendations model. To access BigQuery data from the Colab notebook, you need to upload the service account file that you downloaded in step 10 of the codelab.

In [ ]:
import os
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/content/' + fn
  projectID = fn.rsplit("-", 1)[0]

UsageError: Line magic function `%echo` not found.


# Import app analytics data from BigQuery

In this step, we will load the analytics data we collected in the app with Firebase Analytics and sent to BigQuery. We will load the data into the pandas data processing library and then preprocess this data to be the appropriate format for input for the model training step.

## Enable BigQuery IPython magics

BigQuery provides several convenience IPython magics that we will use to fetch data with the %load_ext magic below.

In [ ]:
%reload_ext google.cloud.bigquery

## Import data

We use the following SQL statement to get items from the table we created in BigQuery. Firebase Analytics exports a lot of additional information, such as device type, platform version, etc, that we don't need for the purposes of training this model. Initially, we only get a limited amount of rows to briefly explore the form of this data and select which fields are important.

Notice that a row in the dataframe is created for each analytics event logged in the app. This row has many properties, but the ones that are of importance for this notebook are the fields:
* event_name
* event_timestamp
* items
* user_pseudo_id

Notice that some fields, such as the **items** field is actually an object. We will extract the subfield of interest below.

In [ ]:
%%bigquery analytics_test_import
SELECT
    *
FROM `firebase_recommendations_dataset.recommendations_table`
LIMIT 10

In [ ]:
analytics_test_import

,items,ecommerce,traffic_source,event_previous_timestamp,user_properties,user_pseudo_id,event_bundle_sequence_id,platform,event_server_timestamp_offset,app_info,event_date,event_name,device,geo,event_params,user_first_touch_timestamp,stream_id,event_timestamp
0,"[{'creative_name': '(not set)', 'item_list_id'...",{'unique_items': 1},"{'medium': '(none)', 'source': '(direct)', 'na...",1598506616201000,[{'value': {'set_timestamp_micros': 1598898000...,3780,41,ANDROID,-3007085,"{'install_source': 'manual_install', 'firebase...",20000811,select_item,"{'time_zone_offset_seconds': 7200, 'operating_...","{'metro': '(not set)', 'sub_continent': 'North...","[{'value': {'string_value': None, 'int_value':...",1597780533660000,2020049249,966032641
1,"[{'creative_name': '(not set)', 'item_list_id'...",{'unique_items': 1},"{'medium': '(none)', 'source': '(direct)', 'na...",1598506616201000,[{'value': {'set_timestamp_micros': 1598898000...,5916,41,ANDROID,-3007085,"{'install_source': 'manual_install', 'firebase...",20000504,select_item,"{'time_zone_offset_seconds': 7200, 'operating_...","{'metro': '(not set)', 'sub_continent': 'North...","[{'value': {'string_value': None, 'int_value':...",1597780533660000,2020049249,957460592
2,"[{'creative_name': '(not set)', 'item_list_id'...",{'unique_items': 1},"{'medium': '(none)', 'source': '(direct)', 'na...",1598506616201000,[{'value': {'set_timestamp_micros': 1598898000...,2303,41,ANDROID,-3007085,"{'install_source': 'manual_install', 'firebase...",20001118,select_item,"{'time_zone_offset_seconds': 7200, 'operating_...","{'metro': '(not set)', 'sub_continent': 'North...","[{'value': {'string_value': None, 'int_value':...",1597780533660000,2020049249,974581189
3,"[{'creative_name': '(not set)', 'item_list_id'...",{'unique_items': 1},"{'medium': '(none)', 'source': '(direct)', 'na...",1598506616201000,[{'value': {'set_timestamp_micros': 1598898000...,4041,41,ANDROID,-3007085,"{'install_source': 'manual_install', 'firebase...",20000805,select_item,"{'time_zone_offset_seconds': 7200, 'operating_...","{'metro': '(not set)', 'sub_continent': 'North...","[{'value': {'string_value': None, 'int_value':...",1597780533660000,2020049249,965511362
4,"[{'creative_name': '(not set)', 'item_list_id'...",{'unique_items': 1},"{'medium': '(none)', 'source': '(direct)', 'na...",1598506616201000,[{'value': {'set_timestamp_micros': 1598898000...,1151,41,ANDROID,-3007085,"{'install_source': 'manual_install', 'firebase...",20001122,select_item,"{'time_zone_offset_seconds': 7200, 'operating_...","{'metro': '(not set)', 'sub_continent': 'North...","[{'value': {'string_value': None, 'int_value':...",1597780533660000,2020049249,974870752
5,"[{'creative_name': '(not set)', 'item_list_id'...",{'unique_items': 1},"{'medium': '(none)', 'source': '(direct)', 'na...",1598506616201000,[{'value': {'set_timestamp_micros': 1598898000...,4335,41,ANDROID,-3007085,"{'install_source': 'manual_install', 'firebase...",20000804,select_item,"{'time_zone_offset_seconds': 7200, 'operating_...","{'metro': '(not set)', 'sub_continent': 'North...","[{'value': {'string_value': None, 'int_value':...",1597780533660000,2020049249,965420431
6,"[{'creative_name': '(not set)', 'item_list_id'...",{'unique_items': 1},"{'medium': '(none)', 'source': '(direct)', 'na...",1598506616201000,[{'value': {'set_timestamp_micros': 1598898000...,1004,41,ANDROID,-3007085,"{'install_source': 'manual_install', 'firebase...",20001124,select_item,"{'time_zone_offset_seconds': 7200, 'operating_...","{'metro': '(not set)', 'sub_continent': 'North...","[{'value': {'string_value': None, 'int_value':...",1597780533660000,2020049249,975052537
7,"[{'creative_name': '(not set)', 'item_list_id'...",{'unique_items': 1},"{'medium': '(none)', 'source': '(direct)', 'na...",1598506616201000,[{'value': {'set_timestamp_micros': 1598898000...,4728,41,ANDROID,-3007085,"{'install_source': 'manual_install', 'firebase...",20000711,select_item,"{'time_zone_offset_seconds': 7200, 'operating_...","{

All of the columns included in each analytics event entry.

In [ ]:
analytics_test_import.columns

Index(['items', 'ecommerce', 'traffic_source', 'event_previous_timestamp',
       'user_properties', 'user_pseudo_id', 'event_bundle_sequence_id',
       'platform', 'event_server_timestamp_offset', 'app_info', 'event_date',
       'event_name', 'device', 'geo', 'event_params',
       'user_first_touch_timestamp', 'stream_id', 'event_timestamp'],
      dtype='object')

Of the information logged under 'items', we are only interested in 'item_id',which corresponds to the ID of the movie the user interacted with.

In [ ]:
analytics_test_import['items'][0][0]

{'affiliation': '(not set)',
 'coupon': '(not set)',
 'creative_name': '(not set)',
 'creative_slot': '(not set)',
 'item_brand': '(not set)',
 'item_category': '(not set)',
 'item_category2': '(not set)',
 'item_category3': '(not set)',
 'item_category4': '(not set)',
 'item_category5': '(not set)',
 'item_id': 256,
 'item_list_id': '(not set)',
 'item_list_index': '(not set)',
 'item_list_name': '(not set)',
 'item_name': 'Movie',
 'item_variant': '(not set)',
 'location_id': '(not set)',
 'promotion_id': '(not set)',
 'promotion_name': '(not set)'}

Now we run the following command to import the whole dataset into a variable. Note how we only import the fields which we are interested in for training purposes.

In [ ]:
%%bigquery analytics_data_real
SELECT
    items,user_pseudo_id,event_timestamp
FROM `firebase_recommendations_dataset.recommendations_table`

In [ ]:
analytics_data_real.head()

,items,user_pseudo_id,event_timestamp
0,"[{'creative_name': '(not set)', 'item_list_id'...",3780,966032641
1,"[{'creative_name': '(not set)', 'item_list_id'...",5916,957460592
2,"[{'creative_name': '(not set)', 'item_list_id'...",2303,974581189
3,"[{'creative_name': '(not set)', 'item_list_id'...",4041,965511362
4,"[{'creative_name': '(not set)', 'item_list_id'...",1151,974870752


# Preprocess the dataset

In this step, we create a lambda function to extract a subfield 'item_id' from the items object. This represents the movie_id, so we also rename the columns to match.

In [ ]:
analytics = analytics_data_real
def getMovieID(row):
  items_obj = row['items'][0]
  return items_obj['item_id']
analytics['movie_id'] = analytics.apply(lambda row: getMovieID(row), axis=1)
analytics

,items,user_pseudo_id,event_timestamp,movie_id
0,"[{'creative_name': '(not set)', 'item_list_id'...",3780,966032641,256
1,"[{'creative_name': '(not set)', 'item_list_id'...",5916,957460592,256
2,"[{'creative_name': '(not set)', 'item_list_id'...",2303,974581189,256
3,"[{'creative_name': '(not set)', 'item_list_id'...",4041,965511362,256
4,"[{'creative_name': '(not set)', 'item_list_id'...",1151,974870752,256
...,...,...,...,...
836473,"[{'creative_name': '(not set)', 'item_list_id'...",3044,970211578,3839
836474,"[{'creative_name': '(not set)', 'item_list_id'...",2852,972507950,3839
836475,"[{'creative_name': '(not set)', 'item_list_id'...",3446,967263885,3839
836476,"[{'creative_name': '(not set)', 'item_list_id'...",2144,974628721,3839


We drop the 'items' column since we don't need anything else from it.

In [ ]:
analytics.rename(columns={'user_pseudo_id': 'user_id', 'event_timestamp': 'timestamp'}, inplace=True)
analytics.drop(['items'], axis=1, inplace=True)

Here is our processed dataframe containing only the data we want to use in training.

The data has the following properties:
*   UserIDs range between 1 and 6040
*   MovieIDs range between 1 and 3952
*   Timestamp is represented in seconds since the epoch as returned by time(2)
*   Each user has at least 20 ratings

In [ ]:
analytics

,user_id,timestamp,movie_id
0,3780,966032641,256
1,5916,957460592,256
2,2303,974581189,256
3,4041,965511362,256
4,1151,974870752,256
...,...,...,...
836473,3044,970211578,3839
836474,2852,972507950,3839
836475,3446,967263885,3839
836476,2144,974628721,3839


## Sort and group training data to create training examples

Our analytics events need to be reorganized in the format required for the model training step. We will create an object that maps key user_id to a list of movies that user has seen. We use the timestamp data to create the sequential context.

In [ ]:
import collections
def convert_to_timelines(df):
  """Convert ratings data to user."""
  timelines = collections.defaultdict(list)
  movie_counts = collections.Counter()
  for user_id, timestamp, movie_id in df.values:
    timelines[user_id].append([movie_id, int(timestamp)])
    movie_counts[movie_id] += 1
  # Sort per-user timeline by timestamp
  for (user_id, timeline) in timelines.items():
    timeline.sort(key=lambda x: x[1])
    timelines[user_id] = [movie_id for movie_id, _ in timeline]
  return timelines, movie_counts
timelines, counts = convert_to_timelines(analytics)

The timelines object contains a list of movie_id's keyed on user_id to indicate the sequence of movies that user has interacted with.

In [ ]:
import itertools

for key, val in sorted(timelines.items())[0:10]:
  print(key, val)

1 [3186, 1721, 1270, 1022, 2340, 1836, 3408, 1207, 2804, 260, 1193, 720, 919, 608, 2692, 1961, 2028, 3105, 938, 1035, 1962, 1028, 150, 2018, 1097, 914, 1287, 2797, 2762, 1246, 661, 2918, 531, 3114, 2791, 1029, 2321, 594, 1197, 2398, 1545, 527, 1, 588, 595, 2687, 745, 783, 2355, 2294, 1566, 1907, 48]
2 [1198, 1210, 1217, 2717, 1293, 2943, 1225, 318, 1193, 2858, 3030, 1945, 1207, 3095, 593, 515, 1873, 3468, 2501, 2067, 110, 3035, 3147, 1247, 3105, 1357, 1196, 1957, 920, 1953, 1834, 1084, 1962, 3654, 3471, 3735, 1954, 1259, 1784, 2728, 1103, 3451, 3334, 2852, 3578, 3068, 265, 2312, 590, 1253, 3071, 1244, 1955, 2236, 3678, 982, 2194, 1442, 2268, 3255, 647, 235, 1096, 1124, 1246, 498, 1537, 1188, 2396, 2321, 2359, 356, 3108, 3809, 1265, 2571, 589, 457, 2028, 1610, 3418, 2916, 380, 163, 480, 349, 1408, 1527, 2353, 2006, 1370, 2278, 648, 1792, 1552, 1372, 780, 1385, 2490, 1801, 2881, 368, 165, 459, 1597, 442, 2628, 1690, 3257, 2002, 736, 292, 2126, 1544, 1917, 1687]
3 [593, 2858, 1968, 3534, 

## Generate training examples

We use the timelines data to generate tensorflow training examples. We discard any timeline with less than 3 context items, and we consider context lengths of 100 items. We perform the following steps:

* Groups movie records by user, and orders per-user movie records by timestamp.
* Generates Tensorflow examples with features: 1) "context": time-ordered sequential movie IDs 2) "label": next movie ID user viewed as label. "max_history_length" is taken in as parameter to define "context" feature shape, if not enough history found, right padding with out-of-vocab ID 0 will be performed.
* Then partition the available data into a training and test set.

Sample generated training example with max user history as 10:
```
0 : {   # (tensorflow.Example)
  features: {   # (tensorflow.Features)
    feature: {
      key  : "context"
      value: {
        int64_list: {
          value: [ 595, 2687, 745, 588, 1, 2355, 2294, 783, 1566, 1907 ]
        }
      }
    }
    feature: {
      key  : "label"
      value: {
        int64_list: {
          value: [ 48 ]
        }
      }
    }
  }
}
```

In [ ]:
import tensorflow as tf

# used to pad when user doesn't have enough context
OOV_MOVIE_ID = 0

def generate_examples_from_timelines(timelines,
                                     min_timeline_len=3,
                                     max_context_len=100):
  """Convert user timelines to tf examples.

  Convert user timelines to tf examples by adding all possible context-label
  pairs in the examples pool.

  Args:
    timelines: the user timelines to process.
    min_timeline_len: minimum length of the user timeline.
    max_context_len: maximum length of context signals.

  Returns:
    train_examples: tf example list for training.
    test_examples: tf example list for testing.
  """
  train_examples = []
  test_examples = []
  for timeline in timelines.values():
    # Skip if timeline is shorter than min_timeline_len.
    if len(timeline) < min_timeline_len:
      continue
    for label_idx in range(1, len(timeline)):
      start_idx = max(0, label_idx - max_context_len)
      context = timeline[start_idx:label_idx]
      # Pad context with out-of-vocab movie id 0.
      while len(context) < max_context_len:
        context.append(OOV_MOVIE_ID)
      label = timeline[label_idx]
      feature = {
          "context":
              tf.train.Feature(int64_list=tf.train.Int64List(value=context)),
          "label":
              tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
      }
      tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
      if label_idx == len(timeline) - 1:
        test_examples.append(tf_example.SerializeToString())
      else:
        train_examples.append(tf_example.SerializeToString())
  return train_examples, test_examples



In [ ]:
train_examples, test_examples = generate_examples_from_timelines(timelines)

Write examples to tfrecords, to be loaded in the model training step.

In [ ]:
def write_tfrecords(tf_examples, filename):
  """Write tf examples to tfrecord file."""
  with tf.io.TFRecordWriter(filename) as file_writer:
    for example in tf_examples:
      file_writer.write(example)

output_dir = 'data/examples'
OUTPUT_TRAINING_DATA_FILENAME = "train_movielens_1m.tfrecord"
OUTPUT_TESTING_DATA_FILENAME = "test_movielens_1m.tfrecord"

if not tf.io.gfile.exists(output_dir):
  tf.io.gfile.makedirs(output_dir)
write_tfrecords(
    tf_examples=train_examples,
    filename=os.path.join(output_dir, OUTPUT_TRAINING_DATA_FILENAME))
write_tfrecords(
    tf_examples=test_examples,
    filename=os.path.join(output_dir, OUTPUT_TESTING_DATA_FILENAME))




# Train model

The training launcher script uses TensorFlow keras compile/fit APIs and performs
the following steps to kick start training and evaluation process:

*   Set up both train and eval dataset input function.
*   Construct keras model according to provided configs, please refer to sample.config file in the source code to config your model architecture, such as embedding dimension, convolutional neural network params, LSTM units etc.
*   Setup loss function. In this code base, we leverages customized batch softmax loss function.
*   Setup optimizer, with flag specified learning rate and gradient clip if needed.
*   Setup evaluation metrics, we provided recall@k metrics by default.
*   Compile model with loss function, optimizer and defined metrics.
*   Setup callbacks for tensorboard and checkpoint manager.
*   Run model.fit with compiled model, where you could specify number of epochs to train, number of train steps in each epoch and number of eval steps in each epoch.

To start training please execute command:


In [ ]:
!python -m model.recommendation_model_launcher_keras \
  --run_mode "train_and_eval" \
  --encoder_type "cnn" \
  --training_data_filepattern "data/examples/train_movielens_1m.tfrecord" \
  --testing_data_filepattern "data/examples/test_movielens_1m.tfrecord" \
  --model_dir "model/model_dir" \
  --params_path "model/sample_config.json"\
  --batch_size 64 \
  --learning_rate 0.1 \
  --steps_per_epoch 1000 \
  --num_epochs 100 \
  --num_eval_steps 1000 \
  --gradient_clip_norm 1.0 \
  --max_history_length 10

2020-10-01 12:47:14.424879: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
INFO:tensorflow:Setting up train and eval input_fns.
I1001 12:47:15.707299 140667743577984 app.py:251] Setting up train and eval input_fns.
INFO:tensorflow:Build keras model for mode: train_and_eval.
I1001 12:47:15.707654 140667743577984 app.py:251] Build keras model for mode: train_and_eval.
2020-10-01 12:47:15.727502: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-01 12:47:15.769101: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-01 12:47:15.769656: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreCloc

# Export model

Now we export the trained model to a tflite file suitable for on-device inference on mobile devices.


In [ ]:
!python -m model.recommendation_model_launcher_keras \
  --run_mode "export" \
  --encoder_type "cnn" \
  --params_path "model/sample_config.json"\
  --model_dir "model/model_dir" \
  --checkpoint_path "model/model_dir/ckpt-100000" \
  --num_predictions 100

2020-10-01 14:52:10.927403: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
INFO:tensorflow:Setting up train and eval input_fns.
I1001 14:52:12.207955 139789516760960 app.py:251] Setting up train and eval input_fns.
INFO:tensorflow:Build keras model for mode: export.
I1001 14:52:12.208148 139789516760960 app.py:251] Build keras model for mode: export.
2020-10-01 14:52:12.214049: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-01 14:52:12.216306: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-01 14:52:12.216846: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz cor

# Model inference (Optional)

You could verify your model's performance by running inference with test examples.

In [ ]:
import tensorflow as tf
import os
import json

# Use [0, 1, ... 9] as example input to represent 10 movies that user interacted with.
#context = [1196, 1210, 2628]
# context = tf.range(10)
context = tf.constant([1196, 1210, 2628, 260, 480, 2571, 589, 1240, 1, 10])

# Directory to exported TensorFlow Lite model.
export_dir = "model/model_dir/export"
tflite_model_path = os.path.join(export_dir, 'model.tflite')
f = open(tflite_model_path, 'rb')
interpreter = tf.lite.Interpreter(model_content=f.read())
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

interpreter.set_tensor(input_details[0]['index'], context)
interpreter.invoke()
tflite_top_predictions_ids = interpreter.get_tensor(
    output_details[0]['index'])
tflite_top_prediction_scores = interpreter.get_tensor(
    output_details[1]['index'])
print("results >>>>>")
print("input >>>>>")
print(input_details[0])
print("output >>>>>")
print(tflite_top_predictions_ids)

[{'name': 'context', 'index': 0, 'shape': array([10], dtype=int32), 'shape_signature': array([10], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 43, 'shape': array([100], dtype=int32), 'shape_signature': array([100], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'Identity_1', 'index': 44, 'shape': array([100], dtype=int32), 'shape_signature': array([100], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
re

# Deploy model to the Firebase Console

We now deploy the model to the Firebase Console. From there, it can be automatically downloaded to your user's devices with Firebase ML. After performing this step, please return to the codelab and complete the last steps.

Step 1. Initialize Firebase App Instance

In [ ]:
import firebase_admin

firebase_admin.initialize_app(
    options={'projectId': projectID, 
             'storageBucket': projectID + '.appspot.com' })

Step 2. Upload the model file to Cloud Storage

In [ ]:
from firebase_admin import ml

# This uploads it to your bucket as recommendation.tflite
source = ml.TFLiteGCSModelSource.from_saved_model(export_dir, 'recommendation.tflite')
print (source.gcs_tflite_uri)

gs://test-5244e.appspot.com/Firebase/ML/Models/recommendation.tflite


Step 3. Deploy the model to Firebase

In [ ]:
# Create a Model Format
model_format = ml.TFLiteFormat(model_source=source)

# Create a Model object
sdk_model_1 = ml.Model(display_name="recommendation", model_format=model_format)

# Make the Create API call to create the model in Firebase
firebase_model_1 = ml.create_model(sdk_model_1)
print(firebase_model_1.as_dict())

# Publish the model
model_id = firebase_model_1.model_id
firebase_model_1 = ml.publish_model(model_id)

{'name': 'projects/test-5244e/models/13692000', 'displayName': 'recommendation', 'createTime': '2020-10-01T14:52:33.183658Z', 'updateTime': '2020-10-01T14:52:33.183658Z', 'state': {}, 'etag': '15ee149656166a0d0e360707e1d770d2cea45a1592687c0688d5358a1409cb61', 'modelHash': '4595bd64315f2e07e8fde86f9e472cc29809a5060df40d77787c945b9a97a4cd', 'tfliteModel': {'sizeBytes': '561196', 'gcsTfliteUri': 'gs://test-5244e.appspot.com/Firebase/ML/Models/recommendation.tflite'}}
